## youtube8Mのデータセット準備

とりあえずローカルでDatasetAPIで学習を開始できるようにするのが目的

### ローカルへのデータダウンロード

youtube8mのデータは以下のコマンドで取って来れます。
下のセルを実行すると全データの1/100のトレーニングデータを取得します。

環境変数からオプションを渡すような作りになってるみたい。
このpartition=2の部分がよくわからないけど、とりあえずこのままでw

In [1]:
# youtube8m のディレクトリに移動
import os
while os.getcwd().split('/')[-1] != 'youtube8m': os.chdir('..')
os.getcwd()

'/home/jupyter/ASLOpenProject/youtube8m'

In [2]:
BUCKET="asl-mixi-project-bucket"

In [3]:
%%bash 
CURRENT_DIR=$(PWD)
mkdir ./data/video/
cd ./data/video/
curl data.yt8m.org/download.py > download.py

shard=1,100 partition=2/video/train mirror=us python download.py
shard=1,100 partition=2/video/validate mirror=us python download.py
cd $CURRENT_DIR

Resuming Download ...
Files remaining 0
All done. No more files to download.
Resuming Download ...
Files remaining 0
All done. No more files to download.


bash: line 1: PWD: command not found
mkdir: cannot create directory ‘./data/video/’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4450  100  4450    0     0   8042      0 --:--:-- --:--:-- --:--:--  8032


In [4]:
!ls ./data/video/train*.tfrecord -l | wc -l

41


In [5]:
!ls ./data/video/train*.tfrecord -alh | head

-rw-r--r-- 1 jupyter jupyter 4.6M Dec  6 08:34 ./data/video/train0093.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.7M Dec  6 08:34 ./data/video/train0111.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.6M Dec  6 08:34 ./data/video/train0208.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.7M Dec  6 08:34 ./data/video/train0274.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.4M Dec  6 08:34 ./data/video/train0276.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0352.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0434.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0477.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0503.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.6M Dec  6 08:34 ./data/video/train0580.tfrecord


In [6]:
!ls ./data/video/validate*.tfrecord -l | wc -l

39


In [7]:
!ls ./data/video/validate*.tfrecord -alh | head

-rw-r--r-- 1 jupyter jupyter 1.5M Dec  7 03:01 ./data/video/validate0024.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.5M Dec  7 03:01 ./data/video/validate0052.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.3M Dec  7 03:01 ./data/video/validate0259.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.3M Dec  7 03:01 ./data/video/validate0267.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.4M Dec  7 03:01 ./data/video/validate0455.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.5M Dec  7 03:01 ./data/video/validate0523.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.4M Dec  7 03:01 ./data/video/validate0638.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.4M Dec  7 03:01 ./data/video/validate0676.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.3M Dec  7 03:01 ./data/video/validate0701.tfrecord
-rw-r--r-- 1 jupyter jupyter 1.3M Dec  7 03:01 ./data/video/validate0780.tfrecord


### storageへのコピー

必要があれば実行。ローカルで実行する分には必要ありません。

In [ ]:
import os
os.environ['BUCKET'] = BUCKET

!gsutil cp ./data/video/*.tfrecord gs://$BUCKET/data/youtube-8m/

### データセットの作成用のコード

In [8]:
import tensorflow as tf

print(tf.__version__)

1.15.0


In [9]:
tf.enable_eager_execution()

In [10]:
from data import read_dataset

In [11]:
ds = read_dataset("./data/video/train*.tfrecord", tf.estimator.ModeKeys.TRAIN, batch_size=2)

In [12]:
for row in ds.take(1):
    print(row)

({'id': <tf.Tensor: id=66, shape=(2,), dtype=string, numpy=array([b'Vm8D', b'V98D'], dtype=object)>, 'mean_rgb': <tf.Tensor: id=68, shape=(2, 1024), dtype=float32, numpy=
array([[ 0.7063481 ,  0.5366782 ,  0.54541487, ...,  0.4460354 ,
        -0.12929349,  0.08991724],
       [-0.8320534 ,  0.8792135 ,  1.6785632 , ...,  0.3766025 ,
        -0.02953384, -0.01896795]], dtype=float32)>, 'mean_audio': <tf.Tensor: id=67, shape=(2, 128), dtype=float32, numpy=
array([[ 9.91503000e-02,  1.65189290e+00,  1.16790187e+00,
         2.71202922e-01,  9.24069643e-01, -9.65530753e-01,
        -6.80397987e-01, -2.82262191e-02,  1.10058987e+00,
        -1.50204110e+00,  9.92870808e-01,  4.78698581e-01,
         5.67355812e-01, -9.99385297e-01, -9.28300679e-01,
         5.52265227e-01,  8.85273144e-02,  9.64873791e-01,
        -9.79430020e-01, -1.77307606e+00,  1.17219299e-01,
        -3.27357531e-01,  8.26697871e-02,  2.71997154e-01,
        -6.55478656e-01,  1.09830642e+00, -8.20879459e-01,
        -